### Introduction
This notebook is designed to help ArcGIS Online / ArcGIS Enterprise administrators manage map areas. Depending on your needs, the manual map area tools in Field Maps Designer and item details may be adequate.  If your managing multiple map areas in multiple maps or having to accomodate frequent changes, a scripted approach may be preferred.  

When should map areas will need to be recreated?

>Re-creating the offline map area differs from updating the offline map area. When you use the Recreate action, it deletes all packages associated with the map area and re-creates them based on the offline map area's settings.
>
>The primary reason to re-create a map area is to pick up schema changes that have occurred after you created the offline map area. For example, if you add or delete a field or change an attribute value list or range (domains), you must re-create the offline map area to pick up those changes.
>
>Source: More info: https://doc.arcgis.com/en/arcgis-online/manage-data/take-maps-offline.htm

This process assumes that you are the owner of web map and that you are using either Enterprise 11.4+ or ArcGIS Online. 

This script can be updated to run on older versions of Enterprise (pre 2.4 ArcGIS API for Python). You would need to update `from arcgis.map import Map` to `from arcgis.mapping import webmap` along with the where webmap is referenced in code. 
More details on ArcGIS API for Python 2.4: https://www.esri.com/arcgis-blog/products/api-python/announcements/whats-new-in-arcgis-api-for-python-2-4-0-blog/


---

### Setup 

The script utilizes three items that will ***need to be created*** and configured in your portal 

#### Hosted Feature Layer 
##### `Manage Map Areas`
- Setup Notes: Download the zipped file geodatabase and add it as an item to your portal.  The hosted feature layer and table below are both included.

    - File hosted on ArcGIS Online: https://www.arcgis.com/home/item.html?id=2ab1be89349e4e61be674fc3e3cbfcdf

    ##### `Manage Map Area Index`
    - Purpose: The polygon hosted feature layer stores map area boundries and processing details.  You can specify if the map area applied to all web map in the group, or a specific web map item id
    - Configuraton: Add your own map area polygons and configure details: 
        - `shape`: Required - this will be the boundary shape for your offline area - map layers will be clipped to this polygon
        - `webmap_id`: Required - Specify which map the offline area should be included in, supports "ALL" web maps or a single web map item id.
        - `status`: Required - Specify if map areas status is "Active" or not if you want to skip over processing.
        - `min_scale`/`max_scale`: Required - these fields are used to identify what level of detail (lods) from the basemaps should be included. The min_scale value is always larger than the max_scale. If you're generating large file sizes for your offline packages, look to refine the max_scale value.
        - `refresh_schedule`: Required - Specifies when the server will sync any changes to map area.  Supports: Never,Daily,Weekly,Monthly.  [More info](https://developers.arcgis.com/python/latest/api-reference/arcgis.map.toc.html#offlinemapareamanager)
        - `refresh_day_of_week`/`refresh_minute`/`refresh_hour`: Optional - Based on choice to `refresh_schedule`  [More info](https://developers.arcgis.com/python/latest/api-reference/arcgis.map.toc.html#offlinemapareamanager)
        - `description`/`title`/`tags`/`snippet`: Optional - for item properties
        - `folder`: Optional - Specify a folder name if you want the offline map area item and the packages to be created inside a folder.
        - More details on fields can be found here: https://developers.arcgis.com/python/latest/api-reference/arcgis.map.toc.html#offlinemapareamanager can be configured.

    ##### `Manage Map Area Processing Log`
    - Purpose: Log table where processing details are stored, this table is used for reporting.
    - Configuration: None needed, the processing script is designed to add rows to this table, but can be configured to truncate for each run as well.
      
#### Group:
##### `Manage Map Areas`
- Purpose: Group should include web maps that will have their map areas processed.  The owner and members of this group will receive email summary of processing results 
- Configuration: Any web map that you own and would like to process map areas for should be added to this group. Any users that you would like to receive processing notification should be included in this group.  This notification is optional

#### After items are created
Copy paste the item ids for `Manage Map Areas` HFL and `Manage Map Areas` group into the `Get Manage Map Areas HFL and Group` section below

--- 

#### Authentication / Security 

The notebook below uses built-in user (`gis = GIS("home")`), which is the account you're logged onto your portal with.  

- For information on different authentication schemes see:
    - https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
- For protecting your credentials see:
    - https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#protecting-your-credentials

---

#### Processing outline: 

The script will create map areas for all web maps in the group `Manage Map Areas` that have an attribute status = 'Active'.  Web maps should include layers and basemap. Basemap can include a reference layer. Proxied basemaps (stored credentials) are supported.  You are required to be owner of each web map

For each web map in group:

1. All existing map areas will be deleted, if it has offline area with status = 'Active'
2. Map areas will be created as defined in hosted feature layer: `Manage Map Area Index`
    - Known limitations:
        - no more than 16 map areas can be defined for the web map
        - a single tile/vector package or mobile database cannot exceed 4 GB
    - Process results will be added to a process log table: `Manage Map Area Processing Log`
    - Reference: https://developers.arcgis.com/python/latest/api-reference/arcgis.map.toc.html#offlinemapareamanager
3. A summary report is emailed to members of `Manage Map Area` group
    - Reference: https://community.esri.com/t5/arcgis-notebooks-documents/send-e-mail-notifications-with-arcgis-notebooks/ta-p/1329699

---

#### Notes on map area download / syncing:
- When a map area is created, a copy of the data is packaged and made available for mobile users to download. This serves as the initial dataset
- The frequency at which the initial dataset refreshes can be defined during the creation of the map area. Please ensure this is set according to the needs of the field operations.
- When field workers download an map area, they store that copy locally on their devices. Any subsequent edits made to the layer's data will be synced back to portal.
- In some circumstances field workers will need to remove and redownload map areas to ensure they have latest changes. 
    - Map areas recreated to include changes to schema changes, field domain changes, new layers
    - Change to web map (form, popup, symbology) 


### Import ArcGIS Libraries and Connect to GIS

In [ ]:
from arcgis.gis import GIS
from arcgis.map import Map # new to 2.4
from arcgis.layers import VectorTileLayer
from arcgis.layers import MapImageLayer
import datetime, time
#from arcgis import env # << uncomment both lines for additional logging detail    
#env.verbose = True

gis = GIS("home")  # see Authentication / Security section above for more detail

### Get Manage Map Areas HFL and Group 

See `Setup` notes above and copy/paste in item ids were shown below

In [ ]:
# ACTION REQUIRED: see SETUP notes above
MAP_AREAS_HFL_ITEM_ID = "42def33b217b4fc8841c3dd7ad00250e" # <<< ACTION REQUIRED: see SETUP notes above to create hosted feature layer on your portal and update item id 
MAP_AREAS_GROUP_ITEM_ID = "797921f945ae4cefb119c932d3ac2571" # <<< ACTION REQUIRED: see SETUP notes above to create group on your portal and update item id 

# get feature layer that contains all the map area polygons
map_areas_hfl = gis.content.get(MAP_AREAS_HFL_ITEM_ID) 
if not map_areas_hfl:
    print("'Manage Map Areas' layer/table could not be found, please check MAP_AREAS_HFL_ITEM_ID...")
    
# get group that has all the shared map area webmaps
manage_map_areas_group = gis.groups.get(MAP_AREAS_GROUP_ITEM_ID)
if not manage_map_areas_group:
    print("'Manage Map Areas' group could not be found, please check MAP_AREAS_GROUP_ITEM_ID...")     

### Helper functions 

In [ ]:
# send report using group notify
def email_group(group_id,subject,message):
    
    try:
        # get group based on passed in id
        notify_group = gis.groups.get(group_id)

        # get group owner and add to list
        notification_list = [notify_group.owner]
        
        # get group members and add them to notification list
        for member in notify_group.get_members()["users"]:
            notification_list.append(member)

        # use group .notify method to send email - message supports html
        notify_result = notify_group.notify(users=notification_list, subject=subject, message=message,method="email")
        
        return notify_result["success"]
        
    except:
        return False

In [ ]:
def create_email_report(map_areas_hfl_item_id, processing_start_time):
    
    try:
        # create html table that includes the query results from Manage Map Area Processing table
        email_subject = ""
        email_body = """<table border='1'>
                            <tr>
                                <th>web map</th>
                                <th>map area name</th>
                                <th>processing status</th>
                                <th>processing seconds</th>
                                <th>total storage mb</th>
                            </tr>"""
        out_fields = "web_map_name, map_area_name,process_status,process_date,process_seconds,total_storage_mb"

        # get logging table
        map_areas_hfl = gis.content.get(map_areas_hfl_item_id)
        logging_table = map_areas_hfl.tables[0]
    
        # email subject - success count 
        subject_success_count_where_clause = f"process_status = 'success' and process_date > '{processing_start_time}'"
        success_results = logging_table.query( where = subject_success_count_where_clause, return_count_only=True )

        # email subject - failed count
        subject_fail_count_where_clause = f"process_status = 'failed' and process_date > '{processing_start_time}'"
        fail_results = logging_table.query(where = subject_fail_count_where_clause,return_count_only=True)   

        # create email subject based on success / fail results
        if success_results > 0 and fail_results == 0:
            email_subject = f"{success_results} map areas created"
        elif success_results > 0 and fail_results > 0:
            email_subject = f"{success_results} map areas created / {fail_results} map areas failed"
        elif success_results == 0 and fail_results > 0:
            email_subject = f"{fail_results} map areas failed"
        else:
            email_subject = "Error processing email subject"
        
        # email body - assemble html table rows     
        where_clause = f"process_date > '{processing_start_time}'"  
        results = logging_table.query(where = where_clause,out_fields = out_fields,sort_field="process_date", sort_order="asc")
        for row in results.features: 
            # output row as we if there is a failure
            row_style = "style='color:#FF0000'" if row.attributes["process_status"] == "failed" else ""
            email_body += f"""<tr {row_style}>
                                <td>{row.attributes["web_map_name"]}</td>
                                <td>{row.attributes["map_area_name"]}</td>
                                <td>{row.attributes["process_status"]}</td>
                                <td>{row.attributes["process_seconds"]}</td>
                                <td>{row.attributes["total_storage_mb"]}</td>
                            </tr>"""                
        email_body += "</table>"
            
        return email_subject, email_body
    
    except Exception as e:
        print(f"an error occurred: {e}")
        return "error creating report","report details not generated"

In [ ]:
def create_map_area_tile_services(baseMapLayers):
    # compile all the base map URLs - your mileage may vary 
    map_area_tile_services = []
    for basemap in baseMapLayers:
        basemap_url = None
        if "url" in basemap:
            # add url
            basemap_url = basemap["url"]

        elif "itemId" in basemap:
            # get items url
            basemap_item = gis.content.get(basemap["itemId"])
            basemap_url = basemap_item["url"]

        elif "styleUrl" in basemap:
            # get source url from 
            vtl = VectorTileLayer(basemap.styleUrl,gis)
            basemap_url = VectorTileLayer(vtl.properties.sources.esri.url,gis).url

        if basemap_url is not None:
            # using the attribute values for min_scale and max_scale - loop through service supported levels and include
            
            try:
                basemap_levels_array = []
                map_service = MapImageLayer(basemap_url,gis)
                for lod in map_service.properties.tileInfo["lods"]:
                    # min_scale and max_scale are required
                    if map_area.get_value("min_scale") >= lod["scale"] >= map_area.get_value("max_scale"):
                        # include this level in basemap_levels
                        basemap_levels_array.append(str(lod["level"]))
                        
            except Exception as e:  
                print("  failed - processing basemap levels")
                print(e)
                
            basemap_levels = ",".join(basemap_levels_array)

            #set the web map tile service parameters url and levels
            map_area_tile_services.append({"url":basemap_url,"levels":basemap_levels})    
        
    return map_area_tile_services



### Processing each web map in group, as defined by index

In [ ]:
print("-- begin processing map areas --")
process_begin_time = datetime.datetime.now()       

# optional - truncate the logs - this can be removed to based on retention preferences
#map_areas_hfl.tables[0].manager.truncate()     

# get all group shared items
for item in manage_map_areas_group.content():
    # if the shared item is not a web map skip it 
    if( item["type"] != "Web Map" ):
        continue # to next item in for loop   

    print("--------------")
    print(f"processing web map: {item['title']}")        

    # Cast item to Map
    web_map = Map(item)

    # create map area as defined in map_areas_hfl
    where_clause =  f"webmap_id IN ('{item.id}', 'ALL') and status = 'Active'"
    map_area_layer = map_areas_hfl.layers[0]
    map_areas_feature_set = map_area_layer.query(where=where_clause) 
    print(f" map area count: {str(len(map_areas_feature_set.features))}")

    # check to ensure there are no more than 16 map areas, if there are skip to next map.
    if len(map_areas_feature_set.features) > 16:
        print(" skipping, more than 16 map areas defined")
        continue # to next item in for loop   

    if len(map_areas_feature_set.features) > 0:
        # delete any existing map areas for map 
        for ids in web_map.offline_areas.list():
            print(f"  - deleting map area: {ids['title']}")
            ids.delete()

    # loop through all the map areas     
    for map_area in map_areas_feature_set.features:
        map_area_start_time = time.time()
        
        # create dictionary for log attributes
        new_processing_log_row = {}
        new_processing_log_row["process_date"] = datetime.datetime.now()
        new_processing_log_row["map_area_name"] = map_area.get_value("title")
        new_processing_log_row["web_map_name"] = item["title"]


        try: 
            # create item properties
            item_prop = {"title": map_area.get_value("title"),
                         "snippet": map_area.get_value("snippet"),
                         "tags": [map_area.get_value("tags")]
                        }

            # call function that processes base map to create tile_services array
            map_area_tile_services = create_map_area_tile_services(web_map.basemap.basemap["baseMapLayers"])
            
            print(f"  requesting map area packaging - {map_area.get_value('title')}")
            map_area_result = web_map.offline_areas.create (area=map_area.geometry,
                                                        item_properties=item_prop,
                                                        refresh_schedule=map_area.get_value("refresh_schedule"),
                                                        refresh_rates = {
                                                          "hour": map_area.get_value("refresh_hour"),
                                                          "minute" : map_area.get_value("refresh_minute"),
                                                          "day_of_week" : map_area.get_value("refresh_day_of_week"),
                                                          "nthday" : map_area.get_value("refresh_day_of_month")
                                                        },
                                                        folder=map_area.get_value("folder"),
                                                        future=False,  # Optional boolean. If True, a future object will be returned and the process will not wait for the task to complete. The default is False, which means wait for results.
                                                        tile_services = map_area_tile_services
                                                     )

            # loop through related items to get the package size in bytes and add it to running total map_area_storage_bytes
            map_area_storage_bytes = 0
            for map_area_pkg in map_area_result.related_items("Area2Package", "forward"):
                if map_area_pkg.size:
                    map_area_storage_bytes += map_area_pkg.size
                    
            # log the map_area_storage_bytes 
            total_storage_mb = round(map_area_storage_bytes / (1000000), 2)
            new_processing_log_row["total_storage_mb"] = total_storage_mb
            # log the process_seconds
            map_area_end_time = time.time()
            new_processing_log_row["process_seconds"] = int(map_area_end_time - map_area_start_time)
            new_processing_log_row["process_status"] = "Success"
            print(f"   + map area packaging complete")
            
        except Exception as e:
            # add row to log
            new_processing_log_row["process_status"] = "Failed"
            print(f"  failed processing map areas {map_area.get_value('title')}")
            print(e)   

        # add the row to logging table successes and failures will be logged
        map_areas_hfl.tables[0].edit_features(adds = [{"attributes": new_processing_log_row}])

print("-- processing map area complete --")      

# create email 
email_subject, email_body = create_email_report(map_areas_hfl.id,process_begin_time)

# email members of Manage Map Areas    
if email_group(manage_map_areas_group.id,email_subject,email_body):
   print("-- report emailed to group members --")
else:
   print("-- report NOT sent to group members --")